In [61]:
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2
from PIL import Image, ImageTk
import customtkinter as ctk
from tkinter import filedialog
from IPython import get_ipython
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


In [62]:
root_dir = 'E:\computer Vision\project\master\Fruit-Images-Dataset-master'
train_dir = "E:\computer Vision\project\master\Fruit-Images-Dataset-master\Training"
test_dir = "E:\computer Vision\project\master\Fruit-Images-Dataset-master\Test"

class_names = sorted(os.listdir(train_dir))
print("All Classes:", class_names)

selected_classes = class_names[:21] 

print("Using classes:", selected_classes)

All Classes: ['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Beetroot', 'Blueberry', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cauliflower', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos', 'Corn', 'Corn Husk', 'Cucumber Ripe', 'Cucumber Ripe 2', 'Dates', 'Eggplant', 'Fig', 'Ginger Root', 'Granadilla', 'Grape Blue', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grape White 3', 'Grape White 4', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Hazelnut', 'Huckleberry', 'Kaki', 'Kiwi', 'Kohlrabi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Mango Red', 'Mangostan', 'Mar

In [63]:

train_files = []
for cls in class_names:
    train_files += glob(os.path.join(train_dir, cls, "*.jpg"))


test_files = []
for cls in class_names:
    test_files += glob(os.path.join(test_dir, cls, "*.jpg"))

print(f"Total training images: {len(train_files)}")
print(f"Total testing images: {len(test_files)}")

Total training images: 67692
Total testing images: 22688


In [64]:
print("Total Classes:", len(class_names))
for i, cls in enumerate(class_names):
    print(f"{i+1}. {cls}")

Total Classes: 131
1. Apple Braeburn
2. Apple Crimson Snow
3. Apple Golden 1
4. Apple Golden 2
5. Apple Golden 3
6. Apple Granny Smith
7. Apple Pink Lady
8. Apple Red 1
9. Apple Red 2
10. Apple Red 3
11. Apple Red Delicious
12. Apple Red Yellow 1
13. Apple Red Yellow 2
14. Apricot
15. Avocado
16. Avocado ripe
17. Banana
18. Banana Lady Finger
19. Banana Red
20. Beetroot
21. Blueberry
22. Cactus fruit
23. Cantaloupe 1
24. Cantaloupe 2
25. Carambula
26. Cauliflower
27. Cherry 1
28. Cherry 2
29. Cherry Rainier
30. Cherry Wax Black
31. Cherry Wax Red
32. Cherry Wax Yellow
33. Chestnut
34. Clementine
35. Cocos
36. Corn
37. Corn Husk
38. Cucumber Ripe
39. Cucumber Ripe 2
40. Dates
41. Eggplant
42. Fig
43. Ginger Root
44. Granadilla
45. Grape Blue
46. Grape Pink
47. Grape White
48. Grape White 2
49. Grape White 3
50. Grape White 4
51. Grapefruit Pink
52. Grapefruit White
53. Guava
54. Hazelnut
55. Huckleberry
56. Kaki
57. Kiwi
58. Kohlrabi
59. Kumquats
60. Lemon
61. Lemon Meyer
62. Limes
63. 

In [65]:
def thresholding_segmentation(image):
    ret2,th2=cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return ret2,th2

In [66]:
class_names = selected_classes
num_classes = len(class_names)
print(f"Selected classes: {class_names}")
print(f"Number of classes: {num_classes}")


Selected classes: ['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Beetroot', 'Blueberry']
Number of classes: 21


In [67]:
def preprocess_image(image):
    image = cv2.resize(image, (64, 64))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    return image


In [68]:
def augment_image(image):
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
    brightness_shift = np.random.randint(-30, 30)
    image = np.clip(image.astype(np.int32) + brightness_shift, 0, 255).astype(np.uint8)
    return image

In [69]:
def load_dataset(root_path, class_names, selected_classes, is_training=False):
    features, labels = [], []
    class_to_idx = {cls: i for i, cls in enumerate(class_names)}

    for class_name in selected_classes:
        class_folder = os.path.join(root_path, class_name)

        for file in tqdm(glob(os.path.join(class_folder, "*.jpg"))):
            image = cv2.imread(file)
            if image is None:
                continue
            if is_training:
                image = augment_image(image)
            image = preprocess_image(image)
            features.append(image)
            labels.append(class_to_idx[class_name])

    
    return np.array(features), np.array(labels)

In [70]:
X_train, y_train = load_dataset(train_dir, class_names, selected_classes, is_training=True)
X_test, y_test = load_dataset(test_dir, class_names, selected_classes, is_training=False)

100%|██████████| 154/154 [00:00<00:00, 2881.87it/s]


In [71]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (8124, 64, 64, 3)
Validation data shape: (2032, 64, 64, 3)
Test data shape: (3395, 64, 64, 3)


In [72]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])



In [73]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

EPOCHS = 10
BATCH_SIZE = 32
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 128)        

In [74]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

107/107 [==============================] - 2s 14ms/step - loss: 0.1364 - accuracy: 0.9549
Test Accuracy: 0.9549


In [75]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_flat, y_train)
y_pred = knn.predict(X_test_flat)
accuracy = accuracy_score(y_test, y_pred)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=selected_classes))

model.save("fruits360_cnn_selected.h5")


Classification Report:
                     precision    recall  f1-score   support

     Apple Braeburn       0.76      0.72      0.74       164
 Apple Crimson Snow       0.77      0.74      0.76       148
     Apple Golden 1       0.97      0.71      0.82       160
     Apple Golden 2       0.87      0.93      0.90       164
     Apple Golden 3       0.83      1.00      0.91       161
 Apple Granny Smith       1.00      1.00      1.00       164
    Apple Pink Lady       0.79      0.98      0.87       152
        Apple Red 1       0.91      0.72      0.80       164
        Apple Red 2       0.76      0.74      0.75       164
        Apple Red 3       1.00      0.65      0.78       144
Apple Red Delicious       0.97      1.00      0.99       166
 Apple Red Yellow 1       1.00      0.98      0.99       164
 Apple Red Yellow 2       1.00      1.00      1.00       219
            Apricot       0.73      0.91      0.81       164
            Avocado       1.00      0.97      0.99       143

c:\Users\Alaa Osman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Alaa Osman\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [76]:
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.87


In [77]:
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
original_image = None
image = None
object_window = None

In [78]:
def on_resize(img):
    return cv2.resize(img, (256, 256))

In [79]:
def on_normalize(img):
    return img / 255.0

In [80]:
def on_noise_reduction(img):
    return cv2.GaussianBlur(img, (5, 5), 0)

In [81]:
def on_contrast_adjustment(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

In [82]:
def on_color_conversion(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [83]:
def on_augmentation_flip(img):
    return cv2.flip(img, 1)

In [84]:
def on_augmentation_rotate(img):
    M = cv2.getRotationMatrix2D((128, 128), 45, 1)
    return cv2.warpAffine(img, M, (256, 256))

In [85]:
def on_augmentation_bright(img):
    return cv2.convertScaleAbs(img, alpha=1.2, beta=30)

In [86]:
def on_thresholding(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    return cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

In [87]:
def on_blur(img):
    return cv2.blur(img, (5, 5))

In [88]:
def on_filtering(img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(img, -1, kernel)

In [89]:
def on_morphology(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5, 5), np.uint8)
    opened = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    return cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR)

In [90]:
def on_eroded(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5, 5), np.uint8)
    eroded = cv2.erode(gray, kernel, iterations=1)
    return cv2.cvtColor(eroded, cv2.COLOR_GRAY2BGR)

In [91]:
def on_dilated(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(gray, kernel, iterations=1)
    return cv2.cvtColor(dilated, cv2.COLOR_GRAY2BGR)

In [92]:
def segment_image(image):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        ret2, segment_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return segment_image, ret2
    except Exception as e:
        raise ValueError(f"Segmentation failed: {str(e)}")

In [93]:
def extract_sift_keypoints(image):
    image_resized = cv2.resize(image, (256, 256))
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(gray, -1, kernel)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(sharpened, None)
    img_keypoints = cv2.drawKeypoints(image_resized, keypoints, None,
                                     flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img_keypoints

In [94]:
def load_image():
    global image, original_image
    
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png ")])
    file_path = os.path.abspath(file_path)
    original_image = cv2.imread(file_path)
    image = original_image.copy()
    display_image(image)

def display_image(img):
    global img_label, root
    img = img.copy()
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((500, 350), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(image=img_pil, master=root)
    img_label.image = img_tk
    img_label.configure(image=img_tk, text="")
    root.update_idletasks()
    root.update()

In [95]:

def display_image(img):
    global img_label, root
    
    img = img.copy()
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((500, 350), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(image=img_pil, master=root)
    img_label.image = img_tk
    img_label.configure(image=img_tk, text="")
    root.update_idletasks()
    root.update()

In [96]:
def display_image_in_new_window(img, img_label):
    global object_window
    
    img = img.copy()
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((250, 150), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(image=img_pil, master=object_window)
    img_label.image = img_tk
    img_label.configure(image=img_tk, text="")
    object_window.update_idletasks()
    object_window.update()
       

In [97]:
def display_image(img):
    global img_label, root
    
    img = img.copy()
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((500, 350), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(image=img_pil, master=root)
    img_label.image = img_tk
    img_label.configure(image=img_tk, text="")
    root.update_idletasks()
    root.update()
        

In [98]:

def display_image_in_new_window(img, img_label):
    global object_window
    img = img.copy()
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((600, 200), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(image=img_pil, master=object_window)
    img_label.image = img_tk
    img_label.configure(image=img_tk, text="")
    object_window.update_idletasks()
    object_window.update()
        

In [99]:
def go_to_object():
    global object_window, root
    object_window = ctk.CTkToplevel(master=root)
    object_window.title("🔍 Object Detection Window")
    object_window.geometry("1500x800")
    object_window.transient(root)
    object_window.protocol("WM_DELETE_WINDOW", lambda: on_object_window_close())

    content_frame = ctk.CTkFrame(object_window, fg_color="#ffffff")
    content_frame.pack(pady=20, padx=20, fill="both", expand=True)

    title_label = ctk.CTkLabel(content_frame, text="Object Detection Processor", font=("Roboto", 16, "bold"))
    title_label.pack(pady=10)

    def load_new_image():
        global object_window
        
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.jpg *.jpeg *.png")],
            parent=object_window
        )
        
        file_path = os.path.abspath(file_path)
        new_image = cv2.imread(file_path)
        error_label.configure(text="")
        display_image_in_new_window(new_image, input_img_label)
        input_img_label.current_image = new_image
        segmented_img_label.configure(image=None, text="Segmented Image")
        keypoints_img_label.configure(image=None, text="SIFT Keypoints")
        classified_img_label.configure(image=None, text="Detected Objects")
        object_window.lift()
        object_window.focus_set()

    browse_btn = ctk.CTkButton(content_frame, text="📁 Load Image", font=("Roboto", 14),
                               command=load_new_image, fg_color="#3498db", hover_color="#2980b9",
                               corner_radius=15, height=40)
    browse_btn.pack(pady=10)

    def detect_image():
        if not hasattr(input_img_label, 'current_image') or input_img_label.current_image is None:
            error_label.configure(text="No image loaded")
            return
        error_label.configure(text="")
    
        segmented_image, keypoints_image, image_with_boxes = process_image(input_img_label.current_image)
        
        display_image_in_new_window(segmented_image, segmented_img_label)
        
        display_image_in_new_window(keypoints_image, keypoints_img_label)
        display_image_in_new_window(image_with_boxes, classified_img_label)
        

    detect_btn = ctk.CTkButton(content_frame, text="Detect", font=("Roboto", 14),
                               command=detect_image, fg_color="#2ecc71", hover_color="#27ae60",
                               corner_radius=15, height=40)
    detect_btn.pack(pady=10)

    error_label = ctk.CTkLabel(content_frame, text="", font=("Roboto", 12), text_color="red")
    error_label.pack(pady=5)

    input_img_label = ctk.CTkLabel(content_frame, text="No image loaded", font=("Roboto", 12))
    input_img_label.pack(pady=10)

    output_frame = ctk.CTkFrame(content_frame, fg_color="transparent")
    output_frame.pack(pady=10, fill="both", expand=True)

    segmented_img_label = ctk.CTkLabel(output_frame, text="Segmented Image", font=("Roboto", 12))
    segmented_img_label.grid(row=0, column=0, padx=10, pady=10)

    keypoints_img_label = ctk.CTkLabel(output_frame, text="SIFT Keypoints", font=("Roboto", 12))
    keypoints_img_label.grid(row=0, column=1, padx=10, pady=10)

    classified_img_label = ctk.CTkLabel(output_frame, text="Detected Objects", font=("Roboto", 12))
    classified_img_label.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky="nsew")


    for i in range(3):
        output_frame.grid_columnconfigure(i, weight=1)
        output_frame.grid_rowconfigure(0, weight=1)
        output_frame.grid_rowconfigure(1, weight=2)  

In [100]:
def preprocess_for_cnn(image):
    image = cv2.resize(image, (64, 64))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    return np.expand_dims(image, axis=0)  

In [101]:
def classify_objects(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = gray / 255.0
    gray_uint8 = (gray * 255).astype(np.uint8)

    
    _, binary_mask = cv2.threshold(gray_uint8, 220, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=2)

    
    dist_transform = cv2.distanceTransform(binary_mask, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.4 * dist_transform.max(), 255, 0)
    sure_fg = sure_fg.astype(np.uint8)

    
    sure_bg = cv2.dilate(binary_mask, kernel, iterations=3)
    unknown = cv2.subtract(sure_bg, sure_fg)

    
    _, markers = cv2.connectedComponents(sure_fg)
    markers += 1
    markers[unknown == 255] = 0
    markers = markers.astype(np.int32)

    
    image_ws = image.copy()
    markers = cv2.watershed(image_ws, markers)
    object_mask = np.uint8(markers > 1)

    
    contours, _ = cv2.findContours(object_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    predicted_classes = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)

       
        if area < 1000:
            continue

        fruit_crop = image[y:y+h, x:x+w]
        input_tensor = preprocess_for_cnn(fruit_crop)

        pred_probs = model.predict(input_tensor, verbose=0)
        pred_index = np.argmax(pred_probs)
        pred_class = selected_classes[pred_index]
        predicted_classes.append(pred_class)
    return predicted_classes, contours


In [102]:
def draw_boxes_and_labels(original_image):
    image = original_image.copy() 
   
    predicted_classes, contours = classify_objects(original_image)
    for idx, cnt in enumerate(contours):
        x, y, w, h = cv2.boundingRect(cnt)

    
        if w < 30 or h < 30:
            continue

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)  


        label = predicted_classes[idx]
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, label, (x, y - 10), font, 0.7, (0, 0, 0), 2)  

    return image

In [103]:
def handle_task(task_name, _=None):
    global original_image, image
    task_map = {
        "Image Resizing": on_resize,
        "Normalization": on_normalize,
        "Noise Reduction": on_noise_reduction,
        "Contrast Adjustment": on_contrast_adjustment,
        "Color Space Conversion": on_color_conversion,
        "Flipping": on_augmentation_flip,
        "Rotation": on_augmentation_rotate,
        "Brightness": on_augmentation_bright,
        "Thresholding": on_thresholding,
        "Blurring": on_blur,
        "Sharpening": on_filtering,
        "Dilation": on_dilated,
        "Erosion": on_eroded,
        "Morphological Operations": on_morphology,
        "SIFT": extract_sift_features,
        "GO": go_to_object,
    }
    task_function = task_map.get(task_name)
    if task_function:
        if task_name == "GO":
            task_function()
        else:
            processed = task_function(original_image.copy())
            image = processed
            display_image(processed)

In [104]:

def process_image(image):
    segmented_image, _ = segment_image(image)
    keypoints_image = extract_sift_keypoints(image)
    
    image_with_boxes = draw_boxes_and_labels(image)

    return segmented_image, keypoints_image, image_with_boxes
    

In [105]:
get_ipython().run_line_magic('gui', 'tk')

root = ctk.CTk()
root.title("🧠 Smart Image Processor")
root.geometry("1500x800")

main_frame = ctk.CTkFrame(root, corner_radius=10, fg_color="#ffffff")
main_frame.place(relx=0.5, rely=0.5, anchor="center", relwidth=0.9, relheight=0.9)

global img_label
img_label = ctk.CTkLabel(main_frame, text="No image loaded", corner_radius=8, fg_color="#f1f3f5")
img_label.pack(pady=15)
img_label.update_idletasks()
img_label.update()

browse_btn = ctk.CTkButton(main_frame, text="📁 Choose Image", font=("Roboto", 14, "bold"),
                           command=load_image, fg_color="#3498db", hover_color="#2980b9",
                           corner_radius=15, height=40)
browse_btn.pack(pady=15)

buttons_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
buttons_frame.pack(pady=20)

tasks = [
    "Image Resizing", "Normalization", "Noise Reduction",
    "Contrast Adjustment", "Color Space Conversion", "Flipping", "Rotation", "Brightness",
    "Thresholding", "Blurring", "Sharpening", "Dilation", "Erosion",
    "Morphological Operations", "SIFT"
]

for idx, task in enumerate(tasks):
    btn = ctk.CTkButton(buttons_frame, text=task, font=("Roboto", 12, "bold"),
                        fg_color="#2ecc71", hover_color="#27ae60", corner_radius=15,
                        height=40, command=lambda t=task: handle_task(t))
    btn.grid(row=idx // 5, column=idx % 5, padx=10, pady=10, sticky="ew")

for i in range(5):
    buttons_frame.grid_columnconfigure(i, weight=1)

go_btn = ctk.CTkButton(main_frame, text="➡ Go to Object Detection Page", font=("Roboto", 18, "bold"),
                       fg_color="#ff6200", hover_color="#cc4e00", corner_radius=20,
                       height=50, width=200, command=lambda: handle_task("GO"),
                       border_width=2, border_color="#ffffff")
go_btn.pack(pady=20)

root.mainloop()

c:\Users\Alaa Osman\AppData\Local\Programs\Python\Python310\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


KeyboardInterrupt: 